# Creating a Sentiment Analysis Web App
### XGBoost AWS SageMaker
_SageMaker, Lambda, API, CloudWatch_

---
Put an overview of the notebook here

## Outline
1. [Download the data](#download)
2. [Process and prepare the data](#process)
3. [Upload data to S3](#upload)
4. [Build and train the Pytorch model](#train)
5. [Test the trained model](#test)
6. [Deploy the trained model](#deploy)
7. [Use the deployed model for inference](#use)


<a id='download'></a>
## Download the Data

The notebook and model use the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/).

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [4]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2021-12-26 15:48:58--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  60.7MB/s    in 1.3s    

2021-12-26 15:48:59 (60.7 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



<a id='process'></a>
## Process and Prepare the Data

---
### Read in Data

In [5]:
# necessary imports 
import os
import glob

In [8]:
def read_imbd_data(data_dir='../data/aclImdb'):
    """ Read in IMDb data from aclImdb folder. Creates data and label dictionaries.
    
        Arguments:
        - data_dir: (str) Directory of the data
        
        Returns:
        - data: (dict) Movie review
        - labels: (dict) Movie review labels
    """
    data = {}
    labels = {}
    
    # create paths to read in review data
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            # join path names
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            # open each review and label. Append to dictionaries and label with binary vars
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                       "{}: data size does not equal {}: label size.".format(data_type, sentiment)
                    
    return data, labels

In [9]:
# read in data and display length of train and test data
data, labels = read_imbd_data()
print("IMDb Reviews: Train = {} pos / {} neg <--> Test = {} pos / {} neg".format(len(data['train']['pos']),
                                                                                    len(data['train']['neg']),
                                                                                    len(labels['test']['pos']),
                                                                                    len(labels['test']['pos'])))

IMDb Reviews: Train = 12500 pos / 12500 neg <--> Test = 12500 pos / 12500 neg


In [10]:
data['train']['pos'][0]

'My family goes back to New Orleans late 1600\'s early 1700\'s and in watching the movie I knew it was a history my grand-parents never talked about, but we knew it existed. I have cousins obviously black aka African Americans and others who can "pass" as white and chose not to. It\'s a hard history to watch when you realize that it\'s your family they\'re talking about and that Cane River is all a part of that history. It makes me want to cry and it makes me want to kick the \'arse\' of my great grandfathers who owned those plantations and wonder in awe of how my great grandmothers of African heritage lived under that oppressive and yet aristocratic existence...And at the same time had I not come out of that history, I probably wouldn\'t be the successful business woman I am today living successfully in a fairly integrated world. The acting was both excellent and fair depending upon the actor, but it is a movie that NEEDED to be made. Anne Rice is incredible and I ask myself, why is s

In [11]:
labels['train']['pos'][0]

1

---
### Create Feature and Target Sets
Combine the training and test data/labels and shuffle to creat feature and target sets.

In [12]:
from sklearn.utils import shuffle

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import re
from bs4 import BeautifulSoup

import pickle

In [13]:
def combine_imdb_data(data, labels):
    """ Combine pos and neg reviews from the training and test data.
        
        Arguments:
        - data: (dict) Unprocessed reviews
        - labels: (dict) Sentiment label, 1 pos --> 0 neg
        
        Returns:
        - train_X, test_X: features
        - train_y, test_y: targets
    """
    # combine positive and negative reviews and labels
    train_data = data['train']['pos'] + data['train']['neg']
    test_data = data['test']['pos'] + data['test']['neg']
    train_labels = labels['train']['pos'] + labels['train']['neg']
    test_labels = labels['test']['pos'] + labels['test']['neg']
    
    # shuffle reviews and labels within training and test sets
    train_data, train_labels = shuffle(train_data, train_labels)
    test_data, test_labels = shuffle(test_data, test_labels)
    
    return train_data, test_data, train_labels, test_labels

In [14]:
train_X, test_X, train_y, test_y = combine_imdb_data(data, labels)
print("IMDb Data Length: Train data = {}, Test data = {}".format(len(train_X), len(test_X)))

IMDb Data Length: Train data = 25000, Test data = 25000


In [15]:
# take a look at a review and it's corresponding label
print(train_X[20], '\n')
print(train_y[20])

This film is very interesting. I have seen it twice and it seems Glover hit the nail on the head with what he claims to he wants to accomplish. I for one can relate to the outrage that the filmmaker clearly expresses against the current thoughtless corporate drivel that is an onslaught in our every media center, and the things that we as a culture are supposed to not "think" about due to corporate media control. The outrage that Glover expresses through the "outrageous" elements in the films is both clear in its visceral aggressiveness and beautiful in its poetic potency. I am glad I saw this film and it is even clearer that Glover is up to something interesting with part two of what will be a trilogy. It is fine! EVERYTHING IS FINE. See that also. People that dismiss this film as "thoughtless" or "pretentious" are really missing the boat. This is an intelligent films. If you can see it with his live show he performs before with his books, that is also very wroth while. The way you get

---
### Process the Data
Remove the html formatting and convert the review into a list of words.

In [16]:
def review_to_words(review):
    """ Converts a review string to a list of words. Removes html
        formatting, stopwords and morphological endings of common
        words.
        
        Arguments:
        - review: (str) String of words that make up review
        
        Returns:
        - words: (list) List of processed words in a review
    
    """ 
    nltk.download('stopwords', quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, 'html.parser').get_text() # remove html tags
    text = re.sub(r"[^a-zA-z0-9]", " ", text.lower()) 
    words = text.split() # split the string into a list of words
    words = [word for word in words if word not in stopwords.words('english')] # remove stopwords
    words = [stemmer.stem(word) for word in words] # stem words
    
    return words


In [17]:
words = review_to_words(train_X[20])
print(words)

['film', 'interest', 'seen', 'twice', 'seem', 'glover', 'hit', 'nail', 'head', 'claim', 'want', 'accomplish', 'one', 'relat', 'outrag', 'filmmak', 'clearli', 'express', 'current', 'thoughtless', 'corpor', 'drivel', 'onslaught', 'everi', 'media', 'center', 'thing', 'cultur', 'suppos', 'think', 'due', 'corpor', 'media', 'control', 'outrag', 'glover', 'express', 'outrag', 'element', 'film', 'clear', 'viscer', 'aggress', 'beauti', 'poetic', 'potenc', 'glad', 'saw', 'film', 'even', 'clearer', 'glover', 'someth', 'interest', 'part', 'two', 'trilog', 'fine', 'everyth', 'fine', 'see', 'also', 'peopl', 'dismiss', 'film', 'thoughtless', 'pretenti', 'realli', 'miss', 'boat', 'intellig', 'film', 'see', 'live', 'show', 'perform', 'book', 'also', 'wroth', 'way', 'get', 'mindset', 'realli', 'someth', 'experi']


In [18]:
cache_dir = os.path.join("../cache", "sentiment_analysis")
os.makedirs(cache_dir, exist_ok=True) 

def preprocess_data(train_data, test_data, train_labels, test_labels,
                    cache_dir=cache_dir, cache_file='preprocesssed_data.pkl'):
    """ Convert each review to words and read from the cache file if 
        available. 
    
    """
    # if cache file exists try to read from it
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f: # open and read binary file
                cache_data = pickle.load(f)
                print("Reading preprocessed data from cache file: {}".format(cache_file))
        except:
            pass
        
    # if cache data does not exist create it
    if cache_data is None:
        # process data to create list of words for each review
        train_words = [review_to_words(review) for review in train_data]
        test_words = [review_to_words(review) for review in test_data]
        
        # write to cache file if it doesn't exist
        if cache_file is not None:
            cache_data = dict(train_words=train_words, test_words=test_words,
                              train_labels=train_labels, test_labels=test_labels)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file: {}".format(cache_file))
            
    else:
        # unpack data from cache file
        train_words = cache_data['train_words']
        test_words = cache_data['test_words']
        train_labels = cache_data['train_labels']
        test_labels = cache_data['test_labels']
        
    return train_words, test_words, train_labels, test_labels

In [23]:
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Reading preprocessed data from cache file: preprocesssed_data.pkl


In [24]:
len(train_X[0])

227

---
### Transform the Data


In [25]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import joblib

In [26]:
def build_dict(data, vocab_size = 5000):
    """ Construct and return a dictionary mapping each of the most 
        frequently appearing words to a unique integer.
        
    """
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for sentance in data:
        for word in sentance:
            if word not in word_count:
                word_count[word] = 1
            else:
                word_count[word] += 1

    sorted_words = sorted(word_count, key=word_count.get, reverse=True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

Take a look at the dictionary to make sure everything looks good. 

In [27]:
word_dict = build_dict(train_X)

In [28]:
most_freq = [key for idx, (key, val) in enumerate(word_dict.items()) if idx < 5]
print(most_freq)

['movi', 'film', 'one', 'like', 'time']


### Save `word_dict`

Later on when we construct an endpoint which processes a submitted review we will need to make use of the `word_dict` which we have created. As such, we will save it to a file now for future use.

In [29]:
data_dir = '../data/pytorch' # folder that will store the data
if not os.path.exists(data_dir): # check if the folder exists
    os.makedirs(data_dir)

In [30]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the Reviews
Convert the reviews into their integer sequence representation. Shorter reviews will be padded with `0` or `1` for no word and infrequent word representation. Longer reviews will be truncated to 500 characters. 

In [32]:
def convert_and_pad(word_dict, review, pad=500):
    NOWORD = 0
    INFREQ = 1
    current_review = [NOWORD] * pad
        
    for idx, word in enumerate(review[:pad]):
        if word in word_dict:
            current_review[idx] = word_dict[word]
        else:
            current_review[idx] = INFREQ
    
    return current_review, min(len(review), pad)

def convert_reviews(word_dict, data, pad=500):
    """ Convert each review to an integer sequence representation. Truncate
        to 500 chars and pad with 0s and 1s accordingly.
        
        Arguments:
        - word_dict: (dict) word mapping dictionary
        - data: reviews
        - pad: (int) length to truncate to
        
        Returns:
        - train_X: feature
        - train_X_len: length of feature set
    """    
    result = []
    lengths = []
    
    for review in data:
        current_review, leng = convert_and_pad(word_dict, review, pad)
        result.append(current_review)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [33]:
train_X, train_X_len = convert_reviews(word_dict, train_X)

In [34]:
test_X, test_X_len = convert_reviews(word_dict, test_X)

In [35]:
print(train_X[0], '\n')
print(train_X_len[20])

[1974 3354    1    1   42 3932  618   91    1  219  279  816   37  207
   89  401   11    1    1    1  122 1803 1386  345   89  175   52  237
 2286 1349 1938  170  471 1352 2566   48  271  345  126 2162 2336   50
 1425    1   50    1   89    1   10   31   62 2829 3845   45 3439    5
  451  816  707   29    1  237 2502  875    1  345  116    6   85  296
   14   37 2502   62  603 2393 4211    1 4632   26   85  105 1707 1205
  273 2143  593  400 1733 1087 1899 1160 2829   78 3439 2829   28   35
   45  202   30 1087 1899   33   63   17 4674 3905  658 2336    1  945
  359 1877  352    1 2991 1425  136 1238  321    1  711   44   72 2272
    1  356  193   52 1279   50  128 3932    1    1   26  193   60  794
 3048 3728 3330   33 1948  489 1345    1    1  384 1342 2503   70   26
  465    2  658  593  400  397   19 1205 1899   60  427  219 4815  633
    1    5   37    1   96   83  208  169 1087  593    1    1   52 2272
    1   29  891 3905  593    1    1  785  344    4   48  192  777   92
 1707 

<a id='upload'></a>
## Upload the Data to S3
Save the data locally and upload to S3 later. Note that the format has to be in the form `label`, `length`, `review`.

### Save and process locally
It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of 500 integers representing the words in the review.

In [36]:
# necessary imports
import pandas as pd
import sagemaker

In [37]:
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Upload the data
Create a SageMaker session, role, and bucket. Upload the data to the default S3 bucket.


In [38]:
# This is an object that represents the SageMaker session that we are currently operating in. This
# object contains some useful information that we will need to access later such as our region.
session = sagemaker.Session()

bucket = session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

# This is an object that represents the IAM role that we are currently assigned. When we construct
# and launch the training job later we will need to tell it what IAM role it should have. Since our
# use case is relatively simple we will simply assign the training job the role we currently have.
role = sagemaker.get_execution_role()

In [39]:
training_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [40]:
print(session.default_bucket())

sagemaker-us-west-2-904606187431


<a id='train'></a>
## Build and Train the Pytorch Model

In [41]:
!pygmentize train/model.py

# PROGRAMMER: Justin Bellucci 
# DATE CREATED: 07_31_2020                                  
# REVISED DATE: 12_23_2021

import torch.nn as nn
import torch

class LSTMClassifier(nn.Module):
    """ LSTM based RNN to perform sentiment analysis
    
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers=2):
        """ Initialize the model by setting up the various 
            layers.
        """
        super(LSTMClassifier, self).__init__()
        
        self.vocab_size = vocab_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers)

        self.fc = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x):
        
        batch_size = x.size(0)
        x = x.t()
        reviews_lengths = x[0,:] # first row is the length of 

Loading in a bit of data to test the model before we use GPU to train on Sagemaker. This is important to identify any mistakes.

In [43]:
# necessary imports
import torch
import torch.utils.data
import torch.nn as nn

import torch.optim as optim


In [44]:
batch_size = 50

# read in the fist 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# turn the Pandas DF into Tensors. Labels are first.
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# build dataset using TensorDataset() from Pytorch
train_sample_dataset = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)

# build the dataloader using DataLoader() from Pytorch
train_sample_loader = torch.utils.data.DataLoader(train_sample_dataset, batch_size=batch_size)

In [45]:
train_sample.transpose()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,1,0,1,1,0,1,1,1,1,1,...,1,0,1,0,0,1,0,1,1,1
1,227,84,89,146,60,332,93,330,108,330,...,19,221,164,141,126,43,123,180,69,41
2,1974,6,69,2863,300,634,7,709,1,1,...,16,537,907,311,979,2,55,2513,180,16
3,3354,2,120,506,174,1050,3,217,1133,1505,...,439,1,3,657,2,620,1,34,1,77
4,1,1,48,356,2,2026,6,177,16,68,...,7,915,211,12,340,72,95,6,261,2668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
500,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
for idx, (x, y) in enumerate(train_sample_loader):
#     print(x.t()[0,:])
    print(x.t()[1:,:])

tensor([[1974,    6,   69,  ...,  356, 1192,   50],
        [3354,    2,  120,  ...,   46, 1260,   10],
        [   1,    1,   48,  ...,   23,  158, 1715],
        ...,
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0]])
tensor([[ 303,  135,  138,  ...,    1,    1,  161],
        [  70,    2,  810,  ..., 1464,  172,   53],
        [ 102,  514,  408,  ..., 3358,  597,  269],
        ...,
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0]])
tensor([[ 487,  135,  581,  ...,  796,   19,  967],
        [   1,    1,    2,  ...,    3,  443, 1950],
        [  12,    3,  497,  ...,  161,  198, 3269],
        ...,
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0]])
tensor([[2494,  815,  

### Training with the small sample dataset

In [47]:
# %load_ext autoreload
# %autoreload 2

from train.model import LSTMClassifier

In [48]:
def train_sample(model, train_loader, epochs, optimizer, criterion, device):
    """ Train a sample dataset in Jupyter notebook.
    """
    for e in range(epochs):
        model.train() # put model in training mode
        total_loss = 0
        
        for batch in train_loader:
            batch_X, batch_y = batch
            
            # move to GPU if available
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
#             h = tuple([each.data for each in h])
            
            # train the model
            optimizer.zero_grad() # zero gradients
            out= model.forward(batch_X)
            
            loss = criterion(out, batch_y)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(e+1, total_loss/len(train_loader)))

In [49]:
embedding_dim = 32
hidden_dim = 100
vocab_size = 5000
epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} to train...".format(device))

model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

train_sample(model, train_sample_loader, epochs, optimizer, criterion, device)

Using cpu to train...
Epoch: 1, BCELoss: 0.6973053932189941
Epoch: 2, BCELoss: 0.6876913785934449
Epoch: 3, BCELoss: 0.6794354557991028
Epoch: 4, BCELoss: 0.6701034545898438
Epoch: 5, BCELoss: 0.6552833437919616


<a id='train'></a>
## Train the Model

In [50]:
# necessary imports
from sagemaker.pytorch import PyTorch
import boto3

In [59]:
pytorch_estimator = PyTorch(entry_point="train.py",
                            source_dir="train",
                            role=role,
                            framework_version='1.0.0',
                            py_version='py3',
                            instance_count=1,
                            instance_type='ml.p2.xlarge',
                            hyperparameters={'epochs': 15,
                                             'hidden_dim': 300})

In [60]:
pytorch_estimator.fit({'training': training_data})

2021-12-26 16:24:09 Starting - Starting the training job...
2021-12-26 16:24:11 Starting - Launching requested ML instancesProfilerReport-1640535848: InProgress
......
2021-12-26 16:25:39 Starting - Preparing the instances for training............
2021-12-26 16:27:39 Downloading - Downloading input data
2021-12-26 16:27:39 Training - Downloading the training image......
2021-12-26 16:28:40 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-26 16:28:25,618 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-12-26 16:28:25,648 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-26 16:28:28,673 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-26 16:28:29,091 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generati

<a id='deploy'></a>
## Deployment

Attach previous training job if necessary.

In [63]:
# training_job_name = "pytorch-training-2021-12-23-18-17-23-313"
# # attach estimator to existing training job
# pytorch_estimator = PyTorch.attach(training_job_name)

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

model_artifacts = pytorch_estimator.model_data
print("\nModel Artifacts: ", model_artifacts)

region = boto_session.region_name
print("Region: ",region)
print("Role: ",role)
print("s3 Bucket: ",bucket)
print("Session: ",session)
print("S3 Train Data Prefix: ",prefix)


Model Artifacts:  s3://sagemaker-us-west-2-904606187431/sagemaker-pytorch-2021-12-26-16-24-08-429/output/model.tar.gz
Region:  us-west-2
Role:  arn:aws:iam::904606187431:role/service-role/AmazonSageMaker-ExecutionRole-20211223T092713
s3 Bucket:  sagemaker-us-west-2-904606187431
Session:  <sagemaker.session.Session object at 0x7f9624eaa1d0>
S3 Train Data Prefix:  sagemaker/sentiment_rnn


### Create the Model

In [72]:
# !pip install -U sagemaker

In [73]:
image_uri = sagemaker.image_uris.retrieve(framework="pytorch",
                                          region=region,
                                          version="1.8.0",
                                          py_version="py3",
                                          image_scope='inference', 
                                          instance_type='ml.m4.4xlarge')

In [76]:
from time import gmtime, strftime

model_name = "pytorch-serverless" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

# # dummy environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20", "SOME_ENV_VAR": "myEnvVar"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

Model name: pytorch-serverless2021-12-26-17-36-41
Model Arn: arn:aws:sagemaker:us-west-2:904606187431:model/pytorch-serverless2021-12-26-17-36-41


### Endpoint Configuration Creation

In [77]:
pytorch_epc_name = "pytorch-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=pytorch_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-west-2:904606187431:endpoint-config/pytorch-serverless-epc2021-12-26-17-36-58


### Serverless Endpoint Creation
Now that we have an endpoint configuration, we can create a serverless endpoint and deploy our model to it. When creating the endpoint, provide the name of your endpoint configuration and a name for the new endpoint.

In [78]:
endpoint_name = "pytorch-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=pytorch_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-west-2:904606187431:endpoint/pytorch-serverless-ep2021-12-26-17-37-58


In [79]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Failed


{'EndpointName': 'pytorch-serverless-ep2021-12-26-17-37-58',
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:904606187431:endpoint/pytorch-serverless-ep2021-12-26-17-37-58',
 'EndpointConfigName': 'pytorch-serverless-epc2021-12-26-17-36-58',
 'EndpointStatus': 'Failed',
 'FailureReason': 'Unable to successfully stand up your model within the allotted 180 second timeout. Please ensure that downloading your model artifacts, starting your model container and passing the ping health checks can be completed within 180 seconds.',
 'CreationTime': datetime.datetime(2021, 12, 26, 17, 37, 58, 444000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 12, 26, 17, 47, 25, 759000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'd43a872c-0610-4c15-b7a2-7b4c1afdc0a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd43a872c-0610-4c15-b7a2-7b4c1afdc0a0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '581',
   'date': 'Sun, 26 Dec 2021 17:47:35 

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

In [130]:
image_uri = pytorch_estimator.training_image_uri()
print(image_uri)

763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.8.0-gpu-py3


In [ ]:
from sagemaker.pytorch import PyTorchModel

# pytorch_model = PyTorchModel(model_data=model_artifacts,
#                              role=role,
#                              framework_version='1.8.0',
#                              source_dir='serve',
#                              entry_point='predict.py',
#                              py_version='py3')



### Model Creation
Create a model by providing your model artifacts, the container image URI, environment variables for the container (if applicable), a model name, and the SageMaker IAM role.

In [132]:
from time import gmtime, strftime

model_name = "pytorch-serverless-" + strftime("%Y-%m-a%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)


# create_model_response = client.create_model(
#     ModelName=model_name,
#     ExecutionRoleArn=role,
#     Containers=[
#         {
#             "Image": pytorch_uri,
#             "Mode": "SingleModel",
#             "ModelDataUrl": model_artifacts,
#         }
#     ],
#     ExecutionRoleArn=role,
# )

# print("Model Arn: " + create_model_response["ModelArn"])

Model name: pytorch-serverless-2021-12-a24-02-32-05


### Endpoint Configuration Creation

This is where you can adjust the <b>Serverless Configuration</b> for your endpoint. The current max concurrent invocations for a single endpoint, known as <b>MaxConcurrency</b>, can be any value from <b>1 to 50</b>, and <b>MemorySize</b> can be any of the following: <b>1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB</b>.

In [133]:
pytorch_epc_name = "pytorch-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=pytorch_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": "pytorch-serverless-2021-12-24-model-01",
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-west-2:904606187431:endpoint-config/pytorch-serverless-epc2021-12-24-02-35-11


### Serverless Endpoint Creation
Now that we have an endpoint configuration, we can create a serverless endpoint and deploy our model to it. When creating the endpoint, provide the name of your endpoint configuration and a name for the new endpoint.

In [134]:
endpoint_name = "pytorch-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=pytorch_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-west-2:904606187431:endpoint/pytorch-serverless-ep2021-12-24-02-35-43


In [135]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Failed


{'EndpointName': 'pytorch-serverless-ep2021-12-24-02-35-43',
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:904606187431:endpoint/pytorch-serverless-ep2021-12-24-02-35-43',
 'EndpointConfigName': 'pytorch-serverless-epc2021-12-24-02-35-11',
 'EndpointStatus': 'Failed',
 'FailureReason': 'Unable to successfully stand up your model within the allotted 180 second timeout. Please ensure that downloading your model artifacts, starting your model container and passing the ping health checks can be completed within 180 seconds.',
 'CreationTime': datetime.datetime(2021, 12, 24, 2, 35, 43, 964000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 12, 24, 2, 45, 57, 215000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'c8e1c04e-7c12-4cd3-bd1e-bc35509e3669',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c8e1c04e-7c12-4cd3-bd1e-bc35509e3669',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '581',
   'date': 'Fri, 24 Dec 2021 02:45:57 GM

<a id='use'></a>
## Use the Deployed Model for Inference

In [ ]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

In [ ]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
def convert_test_review(test_review):
    words = review_to_words(test_review)
    test_data = [np.array(convert_and_pad(word_dict, words)[0])]
    
    return test_data

test_data = convert_test_review(test_review)

In [ ]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=test_data,
    ContentType="text/csv",
)

print(response["Body"].read())